In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [16]:


crypto_df = pd.read_csv(
    Path("../Resources/BitcoinData.csv"),
    index_col='Date', 
    infer_datetime_format=True, 
    parse_dates=True
)


crypto_df.head()



,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2022-10-30,20809.80881,20910.09640,20520.35815,20621.74790,6.473398e+10,3.970800e+11
2022-10-29,20595.37894,21003.55731,20563.93700,20811.98837,7.815063e+10,3.982098e+11
2022-10-28,20292.40585,20733.95162,20074.03700,20608.25790,8.680856e+10,3.907064e+11
2022-10-27,20766.80989,20860.75336,20249.75470,20283.97975,9.798567e+10,3.952218e+11
2022-10-26,20081.82064,20943.08464,20065.93368,20754.02397,1.116419e+11,3.935809e+11


In [6]:
#crypto_df = crypto_df.sort_values(by=['Date'], ascending = True)

In [7]:
crypto_df.fillna(0, inplace=True)

lags = 5

for i in range(0, lags):
    crypto_df['Lag_' + str(i+1)] = crypto_df['Close'].shift(i+1)
    
crypto_df['Returns'] = crypto_df['Close'].pct_change()
    
crypto_df.head(10)

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns
Date,,,,,,,,,,,,
2022-10-30,20809.80881,20910.09640,20520.35815,20621.74790,6.473398e+10,3.970800e+11,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-29,20595.37894,21003.55731,20563.93700,20811.98837,7.815063e+10,3.982098e+11,20621.74790,NaN,NaN,NaN,NaN,0.009225
2022-10-28,20292.40585,20733.95162,20074.03700,20608.25790,8.680856e+10,3.907064e+11,20811.98837,20621.74790,NaN,NaN,NaN,-0.009789
2022-10-27,20766.80989,20860.75336,20249.75470,20283.97975,9.798567e+10,3.952218e+11,20608.25790,20811.98837,20621.74790,NaN,NaN,-0.015735
2022-10-26,20081.82064,20943.08464,20065.93368,20754.02397,1.116419e+11,3.935809e+11,20283.97975,20608.25790,20811.98837,20621.74790,NaN,0.023173
2022-10-25,19330.21290,20388.47250,19252.56340,20110.87556,6.242304e+10,3.754857e+11,20754.02397,20283.97975,20608.25790,20811.98837,20621.74790,-0.030989
2022-10-24,19564.51460,19578.37840,19191.11090,19335.39370,5.756893e+10,3.709005e+11,20110.87556,20754.02397,20283.97975,20608.25790,20811.98837,-0.038560
2022-10-23,19203.79790,19669.50723,19112.11470,19548.72490,2.933448e+10,3.691836e+11,19335.39370,20110.87556,20754.02397,20283.97975,20608.25790,0.011033
2022-10-22,19162.53760,19248.56390,19119.82390,19202.85297,5.115599e+10,3.674651e+11,19548.72490,19335.39370,20110.87556,20754.02397,20283.97975,-0.017693


In [8]:
crypto_df['Direction'] = np.sign(crypto_df['Returns'])

crypto_df = crypto_df.dropna()
crypto_df = crypto_df.drop(crypto_df.index[0:5])

##remove rows with any values that are not infinite
crypto_df = crypto_df[np.isfinite(crypto_df).all(1)]

crypto_df

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns,Direction
Date,,,,,,,,,,,,,
2022-10-20,19123.62910,19328.00080,18942.90900,19042.93090,5.044882e+10,3.664468e+11,19166.3286,19202.85297,19548.72490,19335.39370,20110.87556,-0.006438,-1.0
2022-10-19,19329.17630,19337.07970,19115.23310,19124.98920,6.107692e+10,3.682275e+11,19042.9309,19166.32860,19202.85297,19548.72490,19335.39370,0.004309,1.0
2022-10-18,19546.01540,19685.47556,19116.74420,19351.06970,6.274522e+10,3.732188e+11,19124.9892,19042.93090,19166.32860,19202.85297,19548.72490,0.011821,1.0
2022-10-17,19256.40623,19650.75750,19164.44404,19558.11170,4.874478e+10,3.717332e+11,19351.0697,19124.98920,19042.93090,19166.32860,19202.85297,0.010699,1.0
2022-10-16,19066.05740,19397.58735,19066.05740,19259.62065,3.143498e+10,3.672814e+11,19558.1117,19351.06970,19124.98920,19042.93090,19166.32860,-0.015262,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-11-04,10.80010,10.80010,10.80010,10.80010,0.000000e+00,0.000000e+00,10.7480,10.89900,10.92000,10.92500,10.81500,0.004847,1.0
2012-11-03,10.64250,10.64250,10.64250,10.64250,0.000000e+00,0.000000e+00,10.8001,10.74800,10.89900,10.92000,10.92500,-0.014592,-1.0
2012-11-02,10.46880,10.46880,10.46880,10.46880,0.000000e+00,0.000000e+00,10.6425,10.80010,10.74800,10.89900,10.92000,-0.016321,-1.0


In [9]:
X = crypto_df[['Lag_1', 'Lag_2']]
y = crypto_df['Direction']

crypto_df.head(10)

,Open,High,Low,Close,Volume,Market Cap,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns,Direction
Date,,,,,,,,,,,,,
2022-10-20,19123.62910,19328.00080,18942.90900,19042.93090,5.044882e+10,3.664468e+11,19166.32860,19202.85297,19548.72490,19335.39370,20110.87556,-0.006438,-1.0
2022-10-19,19329.17630,19337.07970,19115.23310,19124.98920,6.107692e+10,3.682275e+11,19042.93090,19166.32860,19202.85297,19548.72490,19335.39370,0.004309,1.0
2022-10-18,19546.01540,19685.47556,19116.74420,19351.06970,6.274522e+10,3.732188e+11,19124.98920,19042.93090,19166.32860,19202.85297,19548.72490,0.011821,1.0
2022-10-17,19256.40623,19650.75750,19164.44404,19558.11170,4.874478e+10,3.717332e+11,19351.06970,19124.98920,19042.93090,19166.32860,19202.85297,0.010699,1.0
2022-10-16,19066.05740,19397.58735,19066.05740,19259.62065,3.143498e+10,3.672814e+11,19558.11170,19351.06970,19124.98920,19042.93090,19166.32860,-0.015262,-1.0
2022-10-15,19182.24760,19219.56470,19026.26720,19054.88470,5.545016e+10,3.668269e+11,19259.62065,19558.11170,19351.06970,19124.98920,19042.93090,-0.010630,-1.0
2022-10-14,19380.65578,19891.32680,19096.98970,19191.02740,9.460731e+10,3.740232e+11,19054.88470,19259.62065,19558.11170,19351.06970,19124.98920,0.007145,1.0
2022-10-13,19155.03423,19454.67640,18290.67140,19380.68930,6.630638e+10,3.647031e+11,19191.02740,19054.88470,19259.62065,19558.11170,19351.06970,0.009883,1.0
2022-10-12,19060.65440,19206.10140,19024.11020,19157.97735,5.722900e+10,3.663985e+11,19380.68930,19191.02740,19054.88470,19259.62065,19558.11170,-0.011491,-1.0


In [10]:
start_test = pd.Timestamp("2012-11-03")

X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]



In [11]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [12]:
lr_accuracy = accuracy_score(y_test, y_pred)
lr_accuracy

0.5424567188788129

In [13]:
pred = (1.0 + (y_pred == y_test))/2.0
hit_rate = np.mean(pred)
print('Logistic Regression - Hit Rate of Bitcoin: {:.4f}'.format(hit_rate))

Logistic Regression - Hit Rate of Bitcoin: 0.7712
